# core

> Main codebase for fetching and saving RSS feeds

In [1]:
#| default_exp core

In [2]:
#| hide
from nbdev.showdoc import *

In [3]:
#| export

def strip_html(s: str):
    s = BeautifulSoup(s)
    return s.text

In [4]:
#| export
import reader
from bs4 import BeautifulSoup
from rich import print
from rich.panel import Panel
from rich.text import Text

reader = reader.make_reader("db.sqlite")

feeds = [
    "https://rss.cbc.ca/lineup/topstories.xml",
    "https://rss.cbc.ca/lineup/world.xml",
    "https://rss.cbc.ca/lineup/canada.xml",
    "https://rss.cbc.ca/lineup/business.xml",
    "https://rss.cbc.ca/lineup/technology.xml",
    "https://www.cbc.ca/cmlink/rss-canada-ottawa",
    "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.topstories.rss",
    "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.news.investigations.rss",
    "https://www.thestar.com/content/thestar/feed.RSSManagerServlet.articles.opinion.editorials.rss",
    "https://www.macleans.ca/feed/",
    "https://thetyee.ca/rss2.xml",
    "https://thenarwhal.ca/feed/",
    "https://ricochet.media/en/feed",
]

for feed in feeds:
    reader.add_feed(feed, exist_ok=True)

## List feeds

In [5]:
[i.title or i.url for i in reader.get_feeds()]

['https://ricochet.media/en/feed',
 'CBC | Business News',
 'CBC | Canada News',
 'CBC | Ottawa News',
 'CBC | Technology News',
 'CBC | Top Stories News',
 'CBC | World News',
 'Macleans.ca',
 'The Narwhal',
 "The Tyee | BC's home for today's news, analysis and updates",
 'TORONTO STAR',
 'TORONTO STAR',
 'TORONTO STAR']

## Update Feeds

In [6]:
reader.update_feeds()

## List Entries

In [7]:
for e in reader.get_entries(read=False, limit=5):
    if e.published:
        published_date = Text(
            "Date: " + e.published.isoformat()[:10], justify="center"
        )
    else:
        published_date = Text("Date: Unknown", justify="center")
    print(
        Panel(
            published_date + "\n" + Text(strip_html(e.summary), justify=None),
            title=f"[link={e.link}]{e.title}[/link]",
            subtitle=e.feed.title or e.feed_url,
        )
    )
    print()

╭────────── ]8;id=960949;https://www.cbc.ca/news/indigenous/afn-assembly-updates-investigations-1.6676651?cmp=rss\AFN assembly kicks off with updates on investigations, little progress on backlogged issues]8;;\ ──────────╮
│                                                Date: 2022-12-07                                                 │
│  The Assembly of First Nations’ first national gathering since July kicked off Tuesday with a pledge to address │
│    the organization’s ongoing internal troubles and an acknowledgment the tensions have resulted in a large     │
│                                           backlog of pressing issues.                                           │
╰──────────────────────────────────────────── CBC | Top Stories News ─────────────────────────────────────────────╯

╭──────────────────────────── ]8;id=971439;https://thetyee.ca/Analysis/2022/12/06/BC-Hasnt-Learned-Much-Pandemic-Report-Covid-Disappointed/\BC Hasn’t Learned Much from the Pandemic (in Analysis)]8;;\ ─────────────────────────────╮
│                                                  Date: Unknown                                                  │
│        A report on the provincial response to the COVID-19 pandemic shows citizens are disappointed and         │
│                                          disenfranchised. Here’s why.                                           │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭────────────────────── ]8;id=190835;https://thetyee.ca/Culture/2022/12/06/Nan-Goldin-Art-Icon-Big-Pharma-Knees/\Meet the Art Icon Who Brought Big Pharma To Its Knees (in Culture)]8;;\ ───────────────────────╮
│                                                  Date: Unknown                                                  │
│                         Art can change the world, and Nan Goldin’s work does just that.                         │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭─────────────────────── ]8;id=9934;https://thetyee.ca/News/2022/12/06/Calls-For-Flu-Fighting-Vaxapalooza/\Calls for a Flu-Fighting ‘Vaxapalooza.’ But What Else? (in News)]8;;\ ────────────────────────╮
│                                                  Date: Unknown                                                  │
│     Beyond the ‘blitz’ of flu shots aimed for by BC health officials, critics urge more focus on masks and      │
│                                                  ventilation.                                                   │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

╭───────────────────── ]8;id=697605;https://thetyee.ca/News/2022/12/06/Die-Without-Home-For-Daughter/\I Don’t Want to Die Without Finding Housing for My Daughter (in News)]8;;\ ─────────────────────╮
│                                                  Date: Unknown                                                  │
│    Aging parents and their adult children with intellectual and developmental disabilities fight for scarce     │
│                                               subsidized rentals.                                               │
╰────────────────────────── The Tyee | BC's home for today's news, analysis and updates ──────────────────────────╯

In [8]:
#| hide
import nbdev

nbdev.nbdev_export()